# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame

In [2]:
df = pd.read_csv('cities.csv')
df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,cherskiy,68.7500,161.3000,33.91,88,100,14.88,RU,1630990669
1,hobart,-42.8794,147.3294,57.06,49,0,4.00,AU,1630990544
2,kaeo,-35.1000,173.7833,59.32,93,89,8.48,NZ,1630990670
3,new norfolk,-42.7826,147.0587,56.71,54,84,5.01,AU,1630990670
4,malinovoye ozero,51.6753,79.7825,74.41,29,21,8.34,RU,1630990671


### Humidity Heatmap

In [3]:
#configue maps 
gmaps.configure(api_key=g_key)

In [4]:
#Use the Lat and Lng as locations and Humidity as the weight.
#Add Heatmap layer to map.
loc=df[['Lat','Lng']]
humidity=df['Humidity']
fig=gmaps.figure(center=(46,10),zoom_level=2)
heat_layer=gmaps.heatmap_layer(loc,weights=humidity,dissipating=False,max_intensity=300,point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria


In [12]:
#Narrow down the cities to fit weather conditions.
#Drop any rows will null values.
hotels = df[(df['Max Temp']>70)&(df['Max Temp']<80)&(df['Wind Speed']<10)&(df['Cloudiness']==0)]
hotels

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,suez,29.9737,32.5263,74.86,68,0,5.44,EG,1630990676
27,karakol,42.4907,78.3936,75.85,18,0,7.18,KG,1630990680
70,kuhdasht,33.5350,47.6061,78.31,12,0,4.47,IR,1630990698
124,le pradet,43.1066,6.0172,72.63,74,0,2.30,FR,1630990699
181,price,39.5994,-110.8107,73.11,20,0,7.58,US,1630990743
250,gela,37.0803,14.2307,70.57,71,0,7.20,IT,1630990773
271,at-bashi,41.1702,75.8106,74.43,22,0,1.57,KG,1630990781
341,portland,45.5234,-122.6762,73.09,55,0,1.01,US,1630990459
425,taywarah,33.3500,64.4167,71.85,11,0,3.53,AF,1630990875
428,beloha,-25.1667,45.0500,72.32,47,0,7.07,MG,1630990876


### Hotel Map

In [14]:
#Store into variable named hotel_df
hotel_df =hotels[['City', 'Country', 'Lat', 'Lng']].copy()

In [15]:
#Add a "Hotel Name" column to the DataFrame
hotel_df['Hotel Name'] = ''

In [8]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
15,suez,EG,29.9737,32.5263,
27,karakol,KG,42.4907,78.3936,
70,kuhdasht,IR,33.5350,47.6061,
124,le pradet,FR,43.1066,6.0172,
181,price,US,39.5994,-110.8107,
250,gela,IT,37.0803,14.2307,
271,at-bashi,KG,41.1702,75.8106,
341,portland,US,45.5234,-122.6762,
425,taywarah,AF,33.3500,64.4167,
428,beloha,MG,-25.1667,45.0500,


In [17]:
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.

def geocode(address):
    '''
    This is function that takes a string address and returns the latitude and longitude
    '''
    gparams = dict(address=address, key=g_key)
   
    # Build the endpoint URL
    target_url = f"https://maps.googleapis.com/maps/api/geocode/json"
    geo_data = requests.get(target_url, params=gparams).json()
    try:
        lat = geo_data["results"][0]["geometry"]["location"]["lat"]
        lng = geo_data["results"][0]["geometry"]["location"]["lng"]
    except:
        lat = None
        lng = None
    return lat, lng

In [20]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_names = requests.get(base_url, params=params)
    
    # print the hotel_names url, avoid doing for public github repos in order to avoid exposing key
    #print(hotel_names.url)
    
    # convert to json
    hotel_names = hotel_names.json()
    #print(json.dumps(hotel_names, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
          hotel_df.loc[index, "Hotel Name"] = hotel_names["results"][0]["name"]
    
    except (KeyError, IndexError):
          print("Missing field/result... skipping.")

Missing field/result... skipping.


In [21]:
#Store the first Hotel result into the DataFrame.
hotel_df

,City,Country,Lat,Lng,Hotel Name
15,suez,EG,29.9737,32.5263,Mövenpick Resort El Sokhna
27,karakol,KG,42.4907,78.3936,Marko Polo
70,kuhdasht,IR,33.5350,47.6061,مهمانپذیر و رستوران ولیعصر (عج) کوهدشت
124,le pradet,FR,43.1066,6.0172,Hôtel Les Roches Blanches
181,price,US,39.5994,-110.8107,Ramada by Wyndham Price
250,gela,IT,37.0803,14.2307,Modica Palace Hotel S
271,at-bashi,KG,41.1702,75.8106,"Отель ""Хан-Тенгри"""
341,portland,US,45.5234,-122.6762,McMenamins Crystal Hotel
425,taywarah,AF,33.3500,64.4167,
428,beloha,MG,-25.1667,45.0500,Le Libertalia


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [23]:
#Plot markers on top of the heatmap.
marker_layer = gmaps.marker_layer(info_box_content=hotel_info, locations=locations)
fig.add_layer(marker_layer)

In [24]:
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))